In [1]:
!pip install datasets
!pip install evaluate
!pip install hopsworks
!pip install jiwer
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.3/170.3 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.3 MB/s eta 0:00:00


In [2]:
import torch
import os
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import DatasetDict, load_dataset
from transformers import WhisperProcessor
from transformers import WhisperTokenizer
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
import evaluate
import hopsworks
from huggingface_hub import login

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open("/content/drive/MyDrive/token.txt", "r") as file:
    token = file.readline().strip()

login(token=token, add_to_git_credential=True)
project = hopsworks.login()

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/193685


In [5]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

metric = evaluate.load("wer")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [6]:
# Hopsworks
# def createDirectoryIfNotExists(directory_name):
#     if not os.path.exists(directory_name):
#         os.mkdir(directory_name)
#
# createDirectoryIfNotExists("common_voice")
# createDirectoryIfNotExists("common_voice/train")
# createDirectoryIfNotExists("common_voice/test")
#
# dataset_api = project.get_dataset_api()
# common_voice = dataset_api.download(
#     "Lab1_ID2222_Training_Datasets/common_voice/train/data-00000-of-00001.arrow", local_path=os.path.dirname(os.getcwd())+"/training/common_voice/train/", overwrite=True)
# common_voice = dataset_api.download(
#     "Lab1_ID2222_Training_Datasets/common_voice/train/dataset_info.json", local_path=os.path.dirname(os.getcwd())+"/training/common_voice/train/", overwrite=True)
# common_voice = dataset_api.download(
#     "Lab1_ID2222_Training_Datasets/common_voice/train/state.json", local_path=os.path.dirname(os.getcwd())+"/training/common_voice/train/", overwrite=True)
#
# common_voice = dataset_api.download(
#     "Lab1_ID2222_Training_Datasets/common_voice/test/data-00000-of-00001.arrow", local_path=os.path.dirname(os.getcwd())+"/training/common_voice/test/", overwrite=True)
# common_voice = dataset_api.download(
#     "Lab1_ID2222_Training_Datasets/common_voice/test/dataset_info.json", local_path=os.path.dirname(os.getcwd())+"/training/common_voice/test/", overwrite=True)
# common_voice = dataset_api.download(
#     "Lab1_ID2222_Training_Datasets/common_voice/test/state.json", local_path=os.path.dirname(os.getcwd())+"/training/common_voice/test/", overwrite=True)
# common_voice = dataset_api.download(
#     "Lab1_ID2222_Training_Datasets/common_voice/dataset_dict.json", local_path=os.path.dirname(os.getcwd())+"/training/common_voice/", overwrite=True)
#
# common_voice = DatasetDict.load_from_disk("common_voice")

In [7]:
#HuggingFace
common_voice = DatasetDict()
common_voice["train"] = load_dataset("dacavi/spanish-dataset", split="train", use_auth_token=True)
common_voice["test"] = load_dataset("dacavi/spanish-dataset", split="test", use_auth_token=True)
print(common_voice)

You can remove this warning by passing 'token=<use_auth_token>' instead.


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/15510 [00:00<?, ? examples/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 15510
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 100
    })
})


In [8]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Spanish", task="transcribe")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Spanish", task="transcribe")
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [9]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

In [10]:
training_args = Seq2SeqTrainingArguments(
    num_train_epochs=1,
    output_dir="dacavi/whisper-small-es",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=1000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_strategy ="steps", #
    save_steps=100, #
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    overwrite_output_dir=False, #
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,

)

In [11]:
processor.save_pretrained(training_args.output_dir)

In [15]:
print("training is starting")
trainer.train(resume_from_checkpoint=True)

training is starting


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


Step,Training Loss,Validation Loss,Wer
500,0.229600,0.237495,17.395833
600,0.264900,0.210192,15.312500
700,0.264400,0.195742,17.395833
800,0.238400,0.188627,13.750000
900,0.232500,0.181148,13.645833
1000,0.137400,0.179750,13.333333


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=1000, training_loss=0.145492267370224, metrics={'train_runtime': 3865.4158, 'train_samples_per_second': 4.139, 'train_steps_per_second': 0.259, 'total_flos': 4.64218474954752e+18, 'train_loss': 0.145492267370224, 'epoch': 1.03})

In [16]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: es, split: test",
    "language": "es",
    "model_name": "Whisper Small Es - Spanish",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

trainer.push_to_hub(**kwargs)

'https://huggingface.co/Dacavi/whisper-small-es/tree/main/'